<a href="https://colab.research.google.com/github/NIDURANGANI/Colab/blob/main/Tutorial_Integration_of_LLMs_using_Langchain_0_3_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain 0.3.27 Tutorial: Real-World Applications with OpenRouter

## Table of Contents
1. [Setup & Installation](#setup--installation)
2. [Basic Configuration](#basic-configuration)
3. [Scenario 1: Customer Support Chatbot](#scenario-1-customer-support-chatbot)
4. [Scenario 2: Document Analysis & Summarization](#scenario-2-document-analysis--summarization)
5. [Scenario 3: Code Review Assistant](#scenario-3-code-review-assistant)
6. [Scenario 4: Content Generation Pipeline](#scenario-4-content-generation-pipeline)
7. [Scenario 5: Data Analysis & Insights](#scenario-5-data-analysis--insights)
8. [Advanced Features](#advanced-features)

## Setup & Installation

### Step 1: Install Required Packages

First, let's install LangChain and required dependencies in your Google Colab environment.

In [51]:
# Install LangChain 0.3.27 and dependencies
!pip install langchain==0.3.27
!pip install langchain-openai
!pip install python-dotenv

In [52]:
# Install additional packages for document processing
!pip install pypdf2
!pip install chromadb
!pip install tiktoken

In [53]:
# Install packages for web scraping and utilities
!pip install beautifulsoup4
!pip install requests
!pip install pandas numpy

In [54]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


### Step 2: Import Required Libraries

In [55]:
import os
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate

In [56]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Basic Configuration

### Step 3: Configure OpenRouter

First, you need to get an API key from [OpenRouter](https://openrouter.ai/). Then set it up in Colab:


In [57]:
# Set your OpenRouter API key (replace with your actual key)
import getpass
api_key = getpass.getpass("Enter your OpenRouter API key: ")
#os.environ["OPENAI_API_KEY"] = "sk-or-v1-d364066ceb3339067def1c2ca462d2f8c5e760aac5b68632b124a7d198ab8bebi"
os.environ["OPENAI_API_KEY"] = api_key

Enter your OpenRouter API key: ··········


In [58]:
# Configure the LLM with OpenRouter
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-oss-20b:free",
    temperature=0.7,
    api_key=os.environ["OPENAI_API_KEY"] # Explicitly pass the API key
)

### Step 4: Test Basic Connection

In [59]:
# Test the connection
response = llm.invoke([HumanMessage(content="Hello! Can you confirm the connection is working?")])
print(response.content)

Hi! Connection is good—everything’s working smoothly on my end. How can I help you today?


## Scenario 1: Customer Support Chatbot

### Real-World Application: E-commerce Customer Service

Let's build a customer support chatbot for an online electronics store.

In [60]:
# Create a system prompt for customer support
system_prompt = """You are a helpful customer support agent for TechStore,
an online electronics retailer. You should be friendly, professional,
and knowledgeable about common issues like orders, returns, and products."""

In [61]:
# Create a conversation memory
memory = ConversationBufferMemory(return_messages=True)

In [62]:
# Create the customer support chain
support_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [63]:
from langchain_core.runnables import RunnableSequence
from operator import itemgetter

# Create the customer support chain with memory integration
# This pattern loads memory, formats the prompt, invokes the LLM, and saves the context
support_chain = (
    {
        "input": itemgetter("input"),
        "chat_history": lambda x: memory.load_memory_variables({})["history"]
    }
    | support_prompt
    | llm
)

# Note: Saving the context back to memory after the response
# is handled separately when invoking the chain in the next step.

In [64]:
# Test the customer support bot
response = support_chain.invoke({"input": "I ordered a laptop 3 days ago but haven't received shipping confirmation"})
print(response.content) # Use .content to access the response text

Hi there haven’t seen a shipping confirmation yet. Let’s get that sorted out right away.

Could you please share your order number (or the email address you used to place the order)? Once I have that, I’ll check the order status and let you know where it is in the process and when you can expect the confirmation email.

If you haven’t received any email at all, I’ll also make sure your contact details are correct and resend the confirmation.

Thanks for your patience—looking forward to helping you!


### Advanced Customer Support Features

In [65]:
# Create a knowledge base for common issues
knowledge_base = {
    "shipping": "Standard shipping takes 3-5 business days. Express shipping takes 1-2 business days.",
    "returns": "You can return items within 30 days of purchase. Items must be in original condition.",
    "warranty": "All electronics come with a 1-year manufacturer warranty."
}

In [66]:
# Enhanced support chain with knowledge lookup
def enhanced_support_response(question):
    # Simple keyword matching for demo
    for key, info in knowledge_base.items():
        if key.lower() in question.lower():
            return f"Based on our policy: {info}\n\nLet me provide more specific help with your question."
    return "Let me help you with that question."

In [67]:
# Test enhanced support
kb_info = enhanced_support_response("What's your return policy?")
full_response = support_chain.invoke({"input": f"{kb_info}\n\nCustomer question: What's your return policy?"}) # Use .invoke()
print(full_response.content) # Use .content to access the response text

Hi there! 👋  
We’re happy to help you with our return policy. Here’s what you need to know:

| **Return Window** | **Timeframe** |
|--------------------|---------------|
| **Standard returns** | 30 days from the date you received the item |
| **Warranty or defect returns** | 60 days from the date of purchase (if the item is defective or not as described) |

### How to Return an Item

1. **Log in to your account** – Go to the “Orders” section and locate the order you’d like to return.  
2. **Start a return** – Click “Return” next to the item.  
3. **Print the return label** – The label is free and includes a barcode for quick processing.  
4. **Pack the item** – Use the original packaging if possible. Include all accessories, manuals, and the original receipt or order confirmation.  
5. **Drop off or schedule a pickup** – Drop the package at any of our partner shipping locations, or schedule a free pickup if you’re in one of the supported regions.

### What to Keep in Mind

- **Conditio

## Scenario 2: Document Analysis & Summarization

### Real-World Application: Legal Document Review

Let's create a system that can analyze and summarize legal contracts or business documents.

In [68]:
# Create sample document content (you can upload files in Colab)
sample_contract = """
EMPLOYMENT AGREEMENT
This Employment Agreement is entered into between TechCorp Inc. and John Smith.
Position: Senior Software Engineer
Start Date: January 15, 2024
Salary: $120,000 annually
Benefits: Health insurance, dental coverage, 401k matching up to 4%
Vacation: 20 days annually
Termination: Either party may terminate with 30 days notice
Non-compete: 6 months restriction in same industry within 50 miles
"""

In [69]:
# Create document analyzer
doc_analyzer_prompt = ChatPromptTemplate.from_template("""
Analyze the following document and provide:
1. Key terms and conditions
2. Important dates and numbers
3. Potential concerns or red flags
4. Brief summary

Document: {document}
""")

In [70]:
doc_chain = LLMChain(llm=llm, prompt=doc_analyzer_prompt)
analysis = doc_chain.run(document=sample_contract)
print(analysis)

**1. Key Terms & Conditions**

| Element | Detail |
|---------|--------|
| **Parties** | TechCorp Inc. (employer) & John Smith (employee) |
| **Position** | Senior Software Engineer |
| **Start Date** | January 15, 2024 |
| **Compensation** | $120,000 per year (annual salary) |
| **Benefits** | • Health insurance<br>• Dental coverage<br>• 401(k) matching up to 4 % of salary |
| **Vacation** | 20 paid days per calendar year |
| **Termination** | Either party may end employment with 30 days written notice |
| **Non‑compete** | 6‑month restriction on working in the same industry within a 50‑mile radius of the employer’s location |

---

**2. Important Dates & Numbers**

| Item | Value |
|------|-------|
| Start Date | Jan 15 2024 |
| Salary | $120,000 / yr |
| 401(k) match | Up to 4 % of salary |
| Vacation days | 20 days/ |
| Notice period | 30 days |
| Non‑compete duration | 6 months |
| Non‑compete radius | 50 miles |

---

**3. Potential Concerns / Red Flags**

| Area | Issue | Why it

### Document Splitting and Processing

In [71]:
# For larger documents, split them into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [72]:
# Create a document summarization chain
summary_prompt = ChatPromptTemplate.from_template("""
Summarize the following text section concisely:
{text}

Summary:
""")

In [73]:
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
chunks = text_splitter.split_text(sample_contract)
summaries = [summary_chain.run(text=chunk) for chunk in chunks]
print("Document summaries:", summaries)

Document summaries: ['**Summary**\n\nTechCorp Inc. hires John Smith as a Senior Software Engineer starting January\u202f15,\u202f2024, with an annual salary of $120,000. Benefits include health, dental, and 401(k) matching up to 4%. Smith receives 20 vacation days per year. Either party may terminate the agreement with 30\u202fdays’ notice. A 6‑month non‑compete applies within the same industry and 50‑mile radius.']


## Scenario 3: Code Review Assistant

### Real-World Application: Automated Code Quality Assessment

Let's build a system that reviews code for best practices, bugs, and improvements.

In [74]:
# Sample code to review
sample_code = '''
def calculate_discount(price, discount_percent):
    discount = price * discount_percent / 100
    final_price = price - discount
    return final_price

# Usage
price = 100
result = calculate_discount(price, 20)
print(f"Final price: ${result}")
'''

In [75]:
# Create code review prompt
code_review_prompt = ChatPromptTemplate.from_template("""
Review the following code and provide:
1. Code quality assessment (1-10)
2. Potential bugs or issues
3. Improvement suggestions
4. Best practices recommendations

Code:
{code}
""")

In [76]:
code_reviewer = LLMChain(llm=llm, prompt=code_review_prompt)
review = code_reviewer.run(code=sample_code)
print(review)

**1. Code quality assessment (1‑10)**  
**Score: 7**

The snippet is clear, concise, and does exactly what it claims to do. However, there are several areas where it could be more robust, self‑documenting, and aligned with Python best practices.

---

## 2. Potential bugs / issues

| # | Issue | Why it matters | Example impact |
|---|-------|----------------|----------------|
| 1 | **No input validation** | `price` or `discount_percent` could be negative, zero, or even non‑numeric. | `calculate_discount(-50, 20)` → negative final price; `calculate_discount(100, 150)` → discount > price. |
| 2 | **Discount percent > 100%** | The function will happily return a negative final price, which is usually unintended. | `calculate_discount(120, 150)` → `-30`. |
| 3 | **Floating‑point precision** | Returning raw float can lead to artifacts like `19.999999999999996` when the user expects `20`. | `calculate_discount(100, 20)` → `80.0` (OK) but more complex numbers may show imprecision. |
| 4 | **No

### Security Code Review

In [77]:
# Security-focused code review
security_prompt = ChatPromptTemplate.from_template("""
Perform a security analysis of this code:
1. Identify security vulnerabilities
2. Check for input validation issues
3. Look for potential injection attacks
4. Suggest security improvements

Code: {code}
""")

In [78]:
security_reviewer = LLMChain(llm=llm, prompt=security_prompt)
security_analysis = security_reviewer.run(code=sample_code)
print(security_analysis)

## Security Analysis of the Provided Code

```python
def calculate_discount(price, discount_percent):
    discount = price * discount_percent / 100
    final_price = price - discount
    return final_price

# Usage
price = 100
result = calculate_discount(price, 20)
print(f"Final price: ${result}")
```

| # | Issue | Why it matters | Severity | Suggested Fix |
|---|-------|----------------|----------|---------------|
| **1** | **No Input Validation** | The function accepts any `price` and `discount_percent`. If a malicious user supplies a string, list, or other non‑numeric value, a `TypeError` will be raised. In a larger application this could crash the service or expose stack traces. | Low | • Add explicit type checks (`isinstance(price, (int, float))`).<br>• Reject or coerce non‑numeric inputs early. <br>• Use `decimal.Decimal` for monetary values. |
| **2** | **Negative or Out‑of‑Range Values** | Negative prices or discounts > 100 % can produce nonsensical results (e.g., a negative f

## Scenario 4: Content Generation Pipeline

### Real-World Application: Marketing Content Creation

Let's create a pipeline that generates marketing content for different platforms.

In [79]:
# Content generation for different platforms
platforms = {
    "twitter": "Create a Twitter post (280 chars max)",
    "linkedin": "Create a LinkedIn professional post",
    "blog": "Create a blog post introduction"
}

In [80]:
# Base content prompt
content_prompt = ChatPromptTemplate.from_template("""
Create {platform_type} content about: {topic}
Target audience: {audience}
Tone: {tone}
Include relevant hashtags where appropriate.
""")

In [81]:
content_generator = LLMChain(llm=llm, prompt=content_prompt)

In [82]:
# Generate content for multiple platforms
topic = "AI in healthcare"
audience = "healthcare professionals"
tone = "professional yet engaging"

In [83]:
# Generate for different platforms
for platform, description in platforms.items():
    content = content_generator.run(
        platform_type=description,
        topic=topic,
        audience=audience,
        tone=tone
    )
    print(f"\n{platform.upper()} CONTENT:\n{content}\n{'-'*50}")


TWITTER CONTENT:
AI is transforming patient care— from predictive analytics to personalized treatment plans. Embrace evidence‑based AI tools to enhance diagnostics, streamline workflows, and improve outcomes. Let’s innovate responsibly. #AIinHealthcare #HealthTech #MedAI #ClinicalData
--------------------------------------------------

LINKEDIN CONTENT:
**AI in Healthcare: A New Frontier for Patient Care & Clinical Efficiency**  

Hello, fellow clinicians, researchers, and health‑tech innovators!  

For years we’ve known that data is the new oil, but it’s only when that oil is refined—by algorithms, machine learning models, and AI‑driven decision support—that we can unlock its full potential. From predictive analytics that flag early sepsis to AI‑assisted imaging that shortens diagnostic turnaround, the impact on patient outcomes is already tangible.

🔍 **What’s driving the shift?**  
- **Big data integration**: Electronic health records, genomics, wearables, and real‑world evidence c

### Content Optimization Chain

In [84]:
# Create content optimization chain
optimization_prompt = ChatPromptTemplate.from_template("""
Optimize this content for better engagement:
1. Make it more compelling
2. Add emotional hooks
3. Include call-to-action
4. Improve readability

Original content: {content}
""")

In [85]:
optimizer = LLMChain(llm=llm, prompt=optimization_prompt)
sample_content = "Our new product uses AI to help doctors."
optimized = optimizer.run(content=sample_content)
print(optimized)

### Meet the Future of Medicine

**Imagine a world where every doctor can diagnose faster, treat smarter, and save more lives—**all thanks to our groundbreaking AI platform.  

- **Speed up diagnoses** by up to 70%  
- **Cut down errors** with real‑time data insights  
- **Give patients personalized care** in minutes, not hours  

Feel the difference when your team can focus on what matters most: *the patient*.  

**Ready to transform your practice?**  
👉 **Sign up for a free demo** today and see the impact for yourself.

---

*Why wait? The future of healthcare is here, and it starts with you.*


## Scenario 5: Data Analysis & Insights

### Real-World Application: Sales Data Analysis

Let's create a system that analyzes sales data and provides business insights.

In [86]:
# Sample sales data
import pandas as pd
sales_data = {
    'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May'],
    'revenue': [50000, 65000, 45000, 70000, 80000],
    'customers': [200, 250, 180, 280, 320]
}
df = pd.DataFrame(sales_data)

In [87]:
# Convert data to text for analysis
data_summary = df.to_string()
print("Sales Data:")
print(data_summary)

Sales Data:
  month  revenue  customers
0   Jan    50000        200
1   Feb    65000        250
2   Mar    45000        180
3   Apr    70000        280
4   May    80000        320


In [88]:
# Create data analysis prompt
analysis_prompt = ChatPromptTemplate.from_template("""
Analyze this sales data and provide:
1. Key trends and patterns
2. Month-over-month growth analysis
3. Customer acquisition insights
4. Recommendations for improvement
5. Predicted next month performance

Data:
{data}
""")

In [89]:
data_analyzer = LLMChain(llm=llm, prompt=analysis_prompt)
insights = data_analyzer.run(data=data_summary)
print(insights)

Below is a quick but thorough look at the first five months of your data.  
All calculations are rounded to two decimals unless otherwise noted.

| Month | Revenue (USD) | Customers | Rev/Customer | MoM Revenue % | MoM Customers % |
|-------|---------------|-----------|--------------|---------------|-----------------|
| Jan   | 50,000 | 200 | 250.00 | – | – |
| Feb   | 65,000 | 250 | 260.00 | **+30.00 %** | **+25.00 %** |
| Mar   | 45,000 | 180 | 250.00 | **‑30.77 %** | **‑28.00 %** |
| Apr   | 70,000 | 280 | 250.00 | **+55.56 %** | **+55.56 %** |
| May   | 80,000 | 320 | 250.00 | **+14.29 %** | **+14.29 %** |

> **Key take‑aways**  
> • Revenue and customers move in lock‑step – the revenue per customer is essentially flat (≈ $250) except for the spike in February.  
> • The data shows a clear “boom‑bust” cycle: a sharp lift in Feb, a dip in Mar, then a rebound in Apr and a moderate lift in May.  
> • The growth rate is volatile: +30 % in Feb, –30.8 % in Mar, +55.6 % in Apr, +14.3 % in

### Automated Report Generation

In [90]:
# Create executive summary generator
report_prompt = ChatPromptTemplate.from_template("""
Create an executive summary report based on this analysis:
{analysis}

Format as:
- Executive Summary (2-3 sentences)
- Key Metrics
- Action Items
- Risk Assessment
""")

In [91]:
report_generator = LLMChain(llm=llm, prompt=report_prompt)
executive_report = report_generator.run(analysis=insights)
print("EXECUTIVE REPORT:")
print(executive_report)

EXECUTIVE REPORT:
**Executive Summary**  
Your business is delivering strong unit economics (≈ $250 per customer) and a clear upward trajectory in revenue and headcount, but the past five months have been highly volatile—peaks in February and April followed by a sharp dip in March.  By addressing the March churn trigger, tightening retention, and capitalising on the April‑May momentum, you can smooth earnings and push the June forecast to roughly **$90 k** in revenue and **380 customers**.

---

### Key Metrics (Jan‑May)

| Metric | Jan‑May | June Forecast | Trend |
|--------|---------|---------------|-------|
| Revenue (USD) | 50 k → 80 k | **$89 500** | +19.6 % MoM avg, +14 % MoM last two months |
| Customers | 200 → 320 | **378** | +16.5 % MoM avg, +14 % MoM last two months |
| Rev/Customer | $250 (stable) | $250 | Unit economics unchanged |
| MoM Revenue Volatility | ±30 % to ±55 % | ±10 % (forecast) | Stabilising with new levers |
| Customer Churn (Mar) | –28 % | 0 % (target) | Re

## Advanced Features

### Chain of Thought Reasoning

In [92]:
# Complex problem-solving with chain of thought
cot_prompt = ChatPromptTemplate.from_template("""
Solve this business problem step by step:
Problem: {problem}

Think through this systematically:
1. First, identify the key issues
2. Then, consider possible solutions
3. Finally, recommend the best approach

Let's work through this step by step:
""")

In [93]:
cot_chain = LLMChain(llm=llm, prompt=cot_prompt)
problem = "Our customer acquisition cost has increased 40% while retention dropped 15%"
solution = cot_chain.run(problem=problem)
print(solution)

## 1️⃣  Identify the Key Issues  

| Area | What’s Happening? | Why It Matters |
|------|------------------|----------------|
| **Customer Acquisition Cost (CAC)** | ↑ 40% in the last 6‑12 months | Higher CAC erodes gross margin, reduces cash‑flow, and limits scalability. |
| **Retention** | ↓ 15% (e.g., churn rate is higher, or repeat‑purchase rate lower) | Lost customers mean lost recurring revenue and higher future CAC (you’ll have to reacquire them). |
| **Marketing Spend Distribution** | Possibly more spend on high‑cost, low‑conversion channels (e.g., paid search, influencer campaigns) | If cost per lead is high and conversion is low, CAC will climb. |
| **Customer Journey** | Longer, more fragmented onboarding; lack of post‑purchase engagement | A friction‑heavy journey increases churn and reduces lifetime value (LTV). |
| **Product/Service Fit** | Feedback indicates feature gaps, usability issues, or mis‑aligned pricing | Product issues often lead to both lower sign‑up conversio

### Multi-Agent Workflow

In [94]:
# Create different "expert" agents
experts = {
    "marketing": "You are a marketing expert focused on customer acquisition and brand strategy.",
    "finance": "You are a finance expert focused on cost optimization and revenue growth.",
    "operations": "You are an operations expert focused on efficiency and process improvement."
}

In [95]:
# Get insights from different experts
def get_expert_opinion(expert_type, question):
    expert_prompt = ChatPromptTemplate.from_template(f"""
    {experts[expert_type]}

    Question: {{question}}

    Provide your expert opinion and specific recommendations:
    """)
    expert_chain = LLMChain(llm=llm, prompt=expert_prompt)
    return expert_chain.run(question=question)

In [96]:
# Consult multiple experts
business_question = "How should we respond to a 20% increase in customer churn?"
for expert in experts.keys():
    opinion = get_expert_opinion(expert, business_question)
    print(f"\n{expert.upper()} EXPERT OPINION:")
    print(opinion)
    print("-" * 60)


MARKETING EXPERT OPINION:
**Executive Summary**

A 20 % jump in churn isn’t just a numbers‑issue; it’s a red flag that something deeper is shifting in the customer journey, the competitive landscape, or the brand promise. To flip the trend from “churn‑driven losses” to “churn‑as‑growth opportunity,” you need a two‑pronged strategy:

| # | Focus Area | What to Do | KPI to Track |
|---|------------|------------|--------------|
| 1 | Diagnostic | Deep churn audit & segmentation | % churn by segment |
| 2 | Retention | Predictive re‑engagement & loyalty | Retention rate, LTV |
| 3 | Product | Feature gap analysis | Feature adoption |
| 4 | Brand | Trust & differentiation | Brand equity, NPS |
| 5 | Acquisition | Re‑balance CAC vs LTV | CAC, ROAS, LTV |

Below is a step‑by‑step playbook you can roll out within 90 days, with measurable milestones.

---

## 1️⃣ Diagnose the Root Causes

### 1.1. Churn Audit
- **Collect & Clean Data**: Pull churned accounts from the last 12 months. Include us

### Error Handling and Validation


In [97]:
def safe_llm_call(chain, **kwargs):
    try:
        result = chain.run(**kwargs)
        return {"success": True, "result": result}
    except Exception as e:
        return {"success": False, "error": str(e)}

In [98]:
# Example with error handling
safe_result = safe_llm_call(content_generator,
                          platform_type="Twitter post",
                          topic="Machine Learning",
                          audience="developers",
                          tone="technical")
print(safe_result)

{'success': True, 'result': '🚀 Building ML models? Start with reproducible pipelines:  \n- Use Docker + GitHub Actions for CI/CD  \n- TensorFlow/PyTorch + JAX for GPU‑accelerated training  \n- MLflow or DVC for experiment tracking  \n- Deploy with FastAPI + Kubernetes  \n\nKeep your code modular, version‑controlled, and test‑first.  \n#MachineLearning #ML #DeepLearning #AI #Python #TensorFlow #PyTorch #JAX #MLOps #DevOps #DataScience 🚀'}


## Best Practices & Tips

### 1. Prompt Engineering Tips
- Be specific and clear in your instructions
- Use examples when possible
- Break complex tasks into smaller steps
- Include context and constraints

### 2. Performance Optimization
- Use appropriate chunk sizes for document processing
- Implement caching for repeated queries
- Monitor token usage to control costs
- Use streaming for real-time applications

### 3. Error Handling
- Always implement try-catch blocks
- Validate inputs before processing
- Have fallback responses ready
- Log errors for debugging

### 4. Security Considerations
- Never expose API keys in code
- Validate and sanitize user inputs
- Implement rate limiting
- Use environment variables for sensitive data